In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from datetime import datetime
import time
import os
import warnings
warnings.filterwarnings("ignore")

######################
# METADATA
######################
sim_start = datetime.now()
sims=1000
input_folder = r'C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\inputs'
input_file = 'inputs_v5.xlsx'
path = os.path.join(input_folder, input_file)

######################
# READ IN DATA
######################
sales_ci=pd.read_excel(path, sheet_name='sales_ci')
unc_pr=pd.read_excel(path, sheet_name='unc_pr')
unc_pr_desc=pd.read_excel(path, sheet_name='unc_pr_desc')
unc_pri=pd.read_excel(path, sheet_name='unc_pri')
unc_pri_desc=pd.read_excel(path, sheet_name='unc_pri_desc')

######################
# PREP DATA
######################
sales_ci=sales_ci.drop_duplicates()
sales_ci=sales_ci.melt(id_vars=['product', 'region', 'units', 'indication'], 
        var_name="year", 
        value_name="sales")
sales_ci=sales_ci[(sales_ci['units'] =='¥')]

# Exclude indications that are not needed
exclude_ind = ['Adjustments', 'Central Adjustments', 'Total']
sales_ci=sales_ci[(~sales_ci['indication'].isin(exclude_ind))]

# Partition WW sales into 9 segments -> US, JP, CN, DE, FR, ES, IT, GB, WWex8
sales_ci_WW=sales_ci[(sales_ci['region']=='WW')]
ast8 = ['US','JP','CN', 'DE','FR','ES','IT','GB']
sales_ci_ast8=sales_ci[(sales_ci['region'].isin(ast8))]

sales_ci_WWex8=sales_ci_ast8.groupby(['product','units','indication','year']).sum().reset_index()
sales_ci_WWex8=sales_ci_WW.merge(sales_ci_WWex8, how='right', on=['product', 'units', 'indication', 'year'])
sales_ci_WWex8['sales']=sales_ci_WWex8['sales_x']-sales_ci_WWex8['sales_y']
sales_ci_WWex8['region']='WWex8'
sales_ci_WWex8 = sales_ci_WWex8[['product','region', 'units', 'indication', 'year', 'sales']]

# Generate new sales table
sales_ci_clean=pd.concat([sales_ci_ast8, sales_ci_WWex8])

# Add back non-strategic products
sales_ci_nsp=sales_ci[(sales_ci['indication']=='Non-Strategic')]
sales_ci_clean=pd.concat([sales_ci_clean, sales_ci_nsp])

# Create tag
sales_ci_clean['tag'] = sales_ci_clean['product'] + sales_ci_clean['region'] + sales_ci_clean['units'] + sales_ci_clean['indication'].astype(str)+ sales_ci_clean['year'].astype(str)

# Validation
print(sum(sales_ci_clean[(sales_ci_clean['year']==2025)]['sales']))  #18491
print(sum(sales_ci_clean[(sales_ci_clean['product']=='xtandi') & (sales_ci_clean['year']==2025)]['sales']))  #7309
print(sum(sales_ci_clean[(sales_ci_clean['product']=='mirabegron') & (sales_ci_clean['year']==2025)]['sales']))  #1223
print(sales_ci_clean['tag'].nunique()) # 920

# Save new sales data
sales_ci_clean.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\sales_ci_clean.csv")

18491.327050511707
7309.662391952652
1223.22
920


In [2]:
#############
# CREATE OUTPUT DFS
#############
output_df = pd.DataFrame(columns=['tag','product', 'region', 'units', 'indication', 'year', 'sales', 'sales_RA_10', 'sales_RA_25', 'sales_RA_50', 'sales_RA_75', 'sales_RA_90']) # Output of Phase 1
scenarios_df = pd.DataFrame(columns=['scenario', 'tag', 'product', 'region', 'units', 'indication', 'year', 'sales', 'sim_sales'])

#############
# PHASE 1: PTRS
#############
# Add PTRS values to sales data
sales_ci_unc = sales_ci_clean.merge(unc_pri, how='right', on=['product','region', 'units', 'indication', 'year'])

scenario_sdf=[]
product_sdf=[]
region_sdf=[]
units_sdf=[]
ind_sdf=[]
year_sdf=[]
sales_sdf=[]
sim_sales_sdf=[]
    
# Loop through rows
for index, row in sales_ci_unc.iterrows():
# for index, row in sales_ci_unc.iloc[29:30].iterrows():
    n=sims
    sales_ra = []
    unc_ptrs_ra = []
    
    for i in range(n):
        scenario_sdf.append(i)
        product_sdf.append(row.values[0])
        region_sdf.append(row.values[1])
        units_sdf.append(row.values[2])
        ind_sdf.append(row.values[3])
        year_sdf.append(row.values[4])
        sales_sdf.append(row.values[5])

        # Get sales (value)
        sales = row.values[5]

        # Get PTRS Uncertainty Probability (value)
        unc_ptrs_prob=np.random.binomial(size=1, n=1, p=row.values[7])
        
        # Get Uncertainty Value (value) for others
        unc1=row.values[8]
        unc2=row.values[9]
        
        if ((unc_ptrs_prob == 1) and ((~np.isnan(unc1)) or (~np.isnan(unc2)) )):
            m=sims
            for j in range(m):

                # Get Uncertainty Probability (value) for others
                unc1_prob=np.random.randint(1,4)
                
                if ((unc1_prob == 1)):
                    sales=unc1
                
                if ((unc1_prob == 2)):
                    sales=unc2
                    
        
        # Generate RA sales
        unc_all=float(sales)*float(unc_ptrs_prob)
        sales_ra.append(unc_all)
        sim_sales_sdf.append(unc_all)
        
    # Get Product Worldwide sales by year (series)
    prod_ww_sales = sales_ci_unc.iloc[index]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
    prod_ww_sales['sales_RA_mean'] = np.mean(sales_ra)
    
    # Append to sales df
    output_df = output_df.append(prod_ww_sales, ignore_index=True)
    
    # Develop scenario_df
    scenarios_df= pd.DataFrame({'scenario': scenario_sdf,'product': product_sdf,'region': region_sdf, 'units': units_sdf, 
                                'indication': ind_sdf, 'year': year_sdf, 'sales': sales_sdf, 'sim_sales': sim_sales_sdf})

# Save outputs
output_df = output_df[['tag','product','region', 'units', 'indication', 'year', 'sales', 'sales_RA_10', 'sales_RA_25', 'sales_RA_50','sales_RA_75', 'sales_RA_90', 'sales_RA_mean']]

sales_exPTRS=sales_ci_clean[~(sales_ci_clean['tag'].isin(output_df['tag']))]
sales_exPTRS['sales_RA_10']=sales_exPTRS['sales']
sales_exPTRS['sales_RA_25']=sales_exPTRS['sales']
sales_exPTRS['sales_RA_50']=sales_exPTRS['sales']
sales_exPTRS['sales_RA_75']=sales_exPTRS['sales']
sales_exPTRS['sales_RA_90']=sales_exPTRS['sales']
sales_exPTRS['sales_RA_mean']=sales_exPTRS['sales']
output_df=pd.concat([output_df, sales_exPTRS])

# Validation
print(sum(output_df[(output_df['year']==2025)]['sales']))  #18491
print(sum(output_df[(output_df['product']=='xtandi') & (output_df['year']==2025)]['sales'])) #7309
print(sum(output_df[(output_df['product']=='mirabegron') & (output_df['year']==2025)]['sales']))  #1223
print(output_df['tag'].nunique()) # 920

output_df.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\output_df_p1.csv")
scenarios_df.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\scenarios_df_p1.csv")

18491.327050511714
7309.662391952652
1223.22
920


In [3]:
#############
# PHASE 2: COMMERCIAL UNCERTAINTY
#############
# Convert PTRS results to sales df for next phase (Commercial Uncertainty)
sales_ptrs=output_df[['tag','product','region', 'units', 'indication', 'year', 'sales', 'sales_RA_mean']]
sales_ptrs.columns = ['tag','product','region', 'units', 'indication', 'year', 'sales', 'sales_p1_RA']

# Sum sales at the WW level
sales_ptrs_ww=sales_ptrs.groupby(['product','units','indication','year']).sum().reset_index()
sales_ptrs_ww['region']='Total'
sales_ptrs=pd.concat([sales_ptrs_ww, sales_ptrs])

# Sum sales at the Product level, as commercial uncertainty is applied at a higher granularity
sales_ptrs_prod=sales_ptrs.groupby(['product','units','region','year']).sum().reset_index()
sales_ptrs_prod['indication']='Total'
sales_ptrs=pd.concat([sales_ptrs_prod, sales_ptrs])

# Validation
print(sum(sales_ptrs[(sales_ptrs['year']==2025)& (sales_ptrs['region']!='Total') & (sales_ptrs['indication']!='Total')]['sales']))  #18491
print(sum(sales_ptrs[(sales_ptrs['product']=='xtandi') & (sales_ptrs['year']==2025) & (sales_ptrs['region']!='Total') & (sales_ptrs['indication']!='Total')]['sales']))  #7309
print(sum(sales_ptrs[(sales_ptrs['product']=='mirabegron') & (sales_ptrs['year']==2025) & (sales_ptrs['region']!='Total') & (sales_ptrs['indication']!='Total')]['sales']))  #1223
print(sales_ptrs[(sales_ptrs['region']!='Total') & (sales_ptrs['indication']!='Total')]['tag'].nunique()) # 920

18491.327050511714
7309.662391952652
1223.22
920


In [4]:
######################
# CREATE INPUT & OUTPUT DF
######################
output_df2 = pd.DataFrame(columns=['tag','product', 'region', 'units', 'indication', 'year', 'sales', 'sales_RA_10', 'sales_RA_25','sales_RA_50','sales_RA_75', 'sales_RA_90'])
output_unc_df = pd.DataFrame(columns=['tag','product', 'region', 'units', 'indication', 'year', 'sales'])

# Create input df
sales_p2a=sales_ptrs[(sales_ptrs['region']=='Total') & (sales_ptrs['indication']=='Total')]
sales_p2b=sales_ptrs[(sales_ptrs['region']!='Total') & (sales_ptrs['indication'].isin(['Total']))]

# Validation
print(sum(sales_p2a[(sales_p2a['year']==2025)]['sales']))  #18491
print(sum(sales_p2a[(sales_p2a['product']=='xtandi') & (sales_p2a['year']==2025)]['sales']))  #7309
print(sum(sales_p2a[(sales_p2a['product']=='mirabegron') & (sales_p2a['year']==2025)]['sales']))  #1223

print(sum(sales_p2b[(sales_p2b['year']==2025)]['sales']))  #18491
print(sum(sales_p2b[(sales_p2b['product']=='xtandi') & (sales_p2b['year']==2025)]['sales']))  #7309
print(sum(sales_p2b[(sales_p2b['product']=='mirabegron') & (sales_p2b['year']==2025)]['sales']))  #1223

18491.32705051171
7309.662391952652
1223.22
18491.3270505117
7309.66239195265
1223.2200000000003


In [5]:
#############
# XTANDI
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'xtandi'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]
output_unc_df=output_unc_df[(output_unc_df['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []
        unc1_ra = []
        unc2_ra = []
        unc3_ra = []
        unc26_ra = []
        
        for i in range(n):
            # Get Uncertainty Probability (value)
            unc1_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc1')]['probability'], 1, 1)
            unc2_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc2')]['probability'], 1, 1)
            unc3_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc3')]['probability'], 1, 1)
            unc26_prob=np.random.binomial(size=1, n=1, p=unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc26')]['probability'])
            
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)
            
            # Get relevant sales values for uncertainties
            unc26_sales=sales_p2b[(sales_p2b['product']==product) & (sales_p2b['year'] ==year) & (sales_p2b['region']==region)]['sales']

            # Get Uncertainty Quant by year (series)
            unc1=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc1']
            unc2=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc2'])*float(unc26_sales)
            unc3=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc3'])*float(unc26_sales)
            unc26=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc26'])*float(unc26_sales)
            
            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob) + float(unc1*unc1_prob) + float(unc2*unc2_prob) + float(unc3*unc3_prob) + float(unc26*unc26_prob)
            sales_ra.append(unc_all)
            
            # Generate uncertainty arrays
            unc1_ra.append(float(unc1*unc1_prob))
            unc2_ra.append(float(unc2*unc2_prob))
            unc3_ra.append(float(unc3*unc3_prob))
            unc26_ra.append(float(unc26*unc26_prob))
            
        # Get Product Region sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
        
        # Add uncertainty arrays to output_unc
        output_unc=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        output_unc['unc1_ra'] = np.mean(unc1_ra)
        output_unc['unc2_ra'] = np.mean(unc2_ra)
        output_unc['unc3_ra'] = np.mean(unc3_ra)
        output_unc['unc26_ra'] = np.mean(unc26_ra)
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
        output_unc_df = output_unc_df.append(output_unc, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,2375.790259,3334.078478,3405.793849,3469.879992,3505.241242,3408.553603,3222.432940
1,0,xtandi,JP,¥,Total,2022,438.632820,425.456861,429.309008,435.738449,441.815860,445.635243,438.632820,435.586629
2,0,xtandi,CN,¥,Total,2022,67.308338,65.170350,65.829296,66.874067,67.878400,68.424932,67.308338,66.844329
3,0,xtandi,DE,¥,Total,2022,518.145979,502.403626,506.973546,514.934678,522.738919,526.640954,518.132556,514.742387
4,0,xtandi,FR,¥,Total,2022,260.011082,251.834907,253.915031,257.698050,262.366899,264.591589,260.011082,258.021722
5,0,xtandi,IT,¥,Total,2022,127.421278,123.367208,124.601067,126.291537,128.244568,129.537218,127.421278,126.427865
6,0,xtandi,ES,¥,Total,2022,99.871451,96.621886,97.470772,99.168844,100.638047,101.560415,99.871451,99.115923
7,0,xtandi,GB,¥,Total,2022,116.406916,112.777071,113.912345,115.611937,117.197722,118.273297,116.406916,115.572478
8,0,xtandi,WWex8,¥,Total,2022,1087.663909,1053.638661,1063.485440,1079.661314,1096.083580,1106.323856,1087.663909,1079.974969
9,0,xtandi,US,¥,Total,2023,3892.422908,2645.110523,3684.672763,3774.272523,3843.170473,3894.518937,3769.573593,3563.579869


In [6]:
print(sum(output_df2[(output_df2['product'] =='xtandi')]['sales']))
print(sum(output_df2[(output_df2['product'] =='xtandi')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='xtandi')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='xtandi')]['sales_RA_75']))

26990.006769473104
25835.917697866284
26380.151718722725
26841.80431755885


In [7]:
#############
# XOSPATA
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'xospata'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]
output_unc_df=output_unc_df[(output_unc_df['product']!=product)]


for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []
        unc4_ra = []
        unc5_ra = []
        unc6_ra = []
        
        for i in range(n):
            # Get Uncertainty Probability (value)
            unc4_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc4')]['probability'], 1, 1)
            unc5_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc5')]['probability'], 1, 1)
            unc6_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc6')]['probability'], 1, 1)

            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Get relevant sales values for uncertainties
            unc4_sales=sales_p2b[(sales_p2b['product']==product) & (sales_p2b['year'] ==year) & (sales_p2b['region']==region)]['sales']
            unc5_sales=sales_p2b[(sales_p2b['product']==product) & (sales_p2b['year'] ==year) & (sales_p2b['region']==region)]['sales']
            unc6_sales=sales_p2b[(sales_p2b['product']==product) & (sales_p2b['year'] ==year) & (sales_p2b['region']==region)]['sales']
            
            # Get Uncertainty Quant by year (series)
            unc4=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc4'])*float(unc4_sales)
            unc5=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc5'])*float(unc5_sales)
            unc6=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc6'])*float(unc6_sales)

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob) + float(unc4*unc4_prob) + float(unc5*unc5_prob) + float(unc6*unc6_prob)
            sales_ra.append(unc_all)
            
            # Generate uncertainty arrays
            unc4_ra.append(float(unc4*unc4_prob))
            unc5_ra.append(float(unc5*unc5_prob))
            unc6_ra.append(float(unc6*unc6_prob))
            
        # Get Product Region sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
        
        # Add uncertainty arrays to output_unc
        output_unc=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        output_unc['unc4_ra'] = np.mean(unc4_ra)
        output_unc['unc5_ra'] = np.mean(unc5_ra)
        output_unc['unc6_ra'] = np.mean(unc6_ra)
        
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
        output_unc_df = output_unc_df.append(output_unc, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,2375.790259,3334.078478,3405.793849,3469.879992,3505.241242,3408.553603,3222.432940
1,0,xtandi,JP,¥,Total,2022,438.632820,425.456861,429.309008,435.738449,441.815860,445.635243,438.632820,435.586629
2,0,xtandi,CN,¥,Total,2022,67.308338,65.170350,65.829296,66.874067,67.878400,68.424932,67.308338,66.844329
3,0,xtandi,DE,¥,Total,2022,518.145979,502.403626,506.973546,514.934678,522.738919,526.640954,518.132556,514.742387
4,0,xtandi,FR,¥,Total,2022,260.011082,251.834907,253.915031,257.698050,262.366899,264.591589,260.011082,258.021722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0,xospata,FR,¥,Total,2025,10.287643,11.179611,11.352197,11.660289,11.958357,12.178670,12.243396,11.662580
68,0,xospata,IT,¥,Total,2025,41.269885,43.146159,43.844424,44.941050,46.162981,46.907946,47.275555,44.990929
69,0,xospata,ES,¥,Total,2025,9.511894,9.144753,9.333677,9.598813,9.849135,9.999412,10.094548,9.588516
70,0,xospata,GB,¥,Total,2025,13.035730,13.536839,13.774742,14.144996,14.516010,14.773721,14.860397,14.147835


In [8]:
print(sum(output_df2[(output_df2['product'] =='xospata')]['sales']))
print(sum(output_df2[(output_df2['product'] =='xospata')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='xospata')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='xospata')]['sales_RA_75']))   

2353.661500188844
2185.3787741710457
2243.4752307714625
2301.940026587112


In [9]:
#############
# PADCEV
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'padcev'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]
output_unc_df=output_unc_df[(output_unc_df['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []
        unc7_ra = []
        unc8_ra = []
        unc9_ra = []
        unc10_ra = []
        unc11_ra = []
        unc12_ra = []
        unc13_ra = []
        unc14_ra = []
        
        for i in range(n):
            # Get Uncertainty Probability (value)
            unc7_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc7')]['probability'], 1, 1)
            unc8_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc8')]['probability'], 1, 1)
            unc9_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc9')]['probability'], 1, 1)
            unc10_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc10')]['probability'], 1, 1)
            unc11_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc11')]['probability'], 1, 1)
            unc12_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc12')]['probability'], 1, 1)
            unc13_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc13')]['probability'], 1, 1)
            unc14_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc14')]['probability'], 1, 1)
        
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Get Uncertainty Quant by year (series)
            unc7=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc7']
            unc8=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc8']
            unc9=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc9']
            unc10=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc10']
            unc11=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc11']
            unc12=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc12']
            unc13=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc13']
            unc14=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc14']

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob) + float(unc7*unc7_prob) + float(unc8*unc8_prob) + float(unc9*unc9_prob) + float(unc10*unc10_prob) + float(unc11*unc11_prob) + float(unc12*unc12_prob) + float(unc13*unc13_prob) + float(unc14*unc14_prob)
            sales_ra.append(unc_all)
            
            # Generate uncertainty arrays
            unc7_ra.append(float(unc7*unc7_prob))
            unc8_ra.append(float(unc8*unc8_prob))
            unc9_ra.append(float(unc9*unc9_prob))
            unc10_ra.append(float(unc10*unc10_prob))
            unc11_ra.append(float(unc11*unc11_prob))
            unc12_ra.append(float(unc12*unc12_prob))
            unc13_ra.append(float(unc13*unc13_prob))
            unc14_ra.append(float(unc14*unc14_prob))

        # Get Product Worldwide sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
        
        # Add uncertainty arrays to output_unc
        output_unc=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        output_unc['unc7_ra'] = np.mean(unc7_ra)
        output_unc['unc8_ra'] = np.mean(unc8_ra)
        output_unc['unc9_ra'] = np.mean(unc9_ra)
        output_unc['unc10_ra'] = np.mean(unc10_ra)
        output_unc['unc11_ra'] = np.mean(unc11_ra)
        output_unc['unc12_ra'] = np.mean(unc12_ra)
        output_unc['unc13_ra'] = np.mean(unc13_ra)
        output_unc['unc14_ra'] = np.mean(unc14_ra)
    
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
        output_unc_df = output_unc_df.append(output_unc, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,2375.790259,3334.078478,3405.793849,3469.879992,3505.241242,3408.553603,3222.432940
1,0,xtandi,JP,¥,Total,2022,438.632820,425.456861,429.309008,435.738449,441.815860,445.635243,438.632820,435.586629
2,0,xtandi,CN,¥,Total,2022,67.308338,65.170350,65.829296,66.874067,67.878400,68.424932,67.308338,66.844329
3,0,xtandi,DE,¥,Total,2022,518.145979,502.403626,506.973546,514.934678,522.738919,526.640954,518.132556,514.742387
4,0,xtandi,FR,¥,Total,2022,260.011082,251.834907,253.915031,257.698050,262.366899,264.591589,260.011082,258.021722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0,padcev,FR,¥,Total,2025,30.210385,10.812352,15.739475,20.789652,25.566556,29.318165,29.976583,20.516507
104,0,padcev,IT,¥,Total,2025,29.306934,25.805691,27.083606,29.326487,31.473370,32.895639,29.287967,29.301139
105,0,padcev,ES,¥,Total,2025,13.922966,12.204090,12.910507,13.891563,14.921729,15.599709,13.893726,13.910404
106,0,padcev,GB,¥,Total,2025,0.492994,0.435841,0.456952,0.494429,0.529210,0.551081,0.492994,0.493848


In [10]:
print(sum(output_df2[(output_df2['product'] =='padcev')]['sales']))
print(sum(output_df2[(output_df2['product'] =='padcev')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='padcev')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='padcev')]['sales_RA_75']))

4084.8039713788817
3733.2758310843815
4063.6745909505316
4390.507396615152


In [11]:
#############
# EVRENZO
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'evrenzo'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]
output_unc_df=output_unc_df[(output_unc_df['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []
        unc15_ra = []
        unc16_ra = []
        unc17_ra = []
        unc18_ra = []
        
        for i in range(n):
            # Get Uncertainty Probability (value)
            unc15_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc15')]['probability'], 1, 1)
            unc16_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc16')]['probability'], 1, 1)
            unc17_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc17')]['probability'], 1, 1)
            unc18_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc18')]['probability'], 1, 1)
        
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Get Uncertainty Quant by year (series)
            unc15=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc15']
            unc16=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc16']
            unc17=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc17']
            unc18=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc18']

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob) + float(unc15*unc15_prob) + float(unc16*unc16_prob) + float(unc17*unc17_prob) + float(unc18*unc18_prob)
            sales_ra.append(unc_all)
            
            # Generate uncertainty arrays
            unc15_ra.append(float(unc15*unc15_prob))
            unc16_ra.append(float(unc16*unc16_prob))
            unc17_ra.append(float(unc17*unc17_prob))
            unc18_ra.append(float(unc18*unc18_prob))

        # Get Product Worldwide sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
        
        # Add uncertainty arrays to output_unc
        output_unc=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        output_unc['unc15_ra'] = np.mean(unc15_ra)
        output_unc['unc16_ra'] = np.mean(unc16_ra)
        output_unc['unc17_ra'] = np.mean(unc17_ra)
        output_unc['unc18_ra'] = np.mean(unc18_ra)
        
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
        output_unc_df = output_unc_df.append(output_unc, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,2375.790259,3334.078478,3405.793849,3469.879992,3505.241242,3408.553603,3222.432940
1,0,xtandi,JP,¥,Total,2022,438.632820,425.456861,429.309008,435.738449,441.815860,445.635243,438.632820,435.586629
2,0,xtandi,CN,¥,Total,2022,67.308338,65.170350,65.829296,66.874067,67.878400,68.424932,67.308338,66.844329
3,0,xtandi,DE,¥,Total,2022,518.145979,502.403626,506.973546,514.934678,522.738919,526.640954,518.132556,514.742387
4,0,xtandi,FR,¥,Total,2022,260.011082,251.834907,253.915031,257.698050,262.366899,264.591589,260.011082,258.021722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,0,evrenzo,FR,¥,Total,2025,46.025024,38.852416,41.168578,44.443172,48.231907,50.355915,46.025024,44.542790
140,0,evrenzo,IT,¥,Total,2025,72.868764,45.033748,50.306999,56.344663,62.891613,68.460866,72.868764,56.737162
141,0,evrenzo,ES,¥,Total,2025,14.194413,10.370193,11.381378,12.475600,13.604525,14.452418,14.194413,12.459208
142,0,evrenzo,GB,¥,Total,2025,17.100499,15.047567,15.889059,17.088355,18.338254,19.173700,17.100499,17.118806


In [12]:
print(sum(output_df2[(output_df2['product'] =='evrenzo')]['sales']))
print(sum(output_df2[(output_df2['product'] =='evrenzo')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='evrenzo')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='evrenzo')]['sales_RA_75']))

1113.6799883146336
1012.9844492209916
1100.427654222935
1186.927900251601


In [13]:
#############
# ZOLBE
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'zolbe'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]
output_unc_df=output_unc_df[(output_unc_df['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []
        
        for i in range(n):
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob)
            sales_ra.append(unc_all)

        # Get Product Worldwide sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
    
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,2375.790259,3334.078478,3405.793849,3469.879992,3505.241242,3408.553603,3222.432940
1,0,xtandi,JP,¥,Total,2022,438.632820,425.456861,429.309008,435.738449,441.815860,445.635243,438.632820,435.586629
2,0,xtandi,CN,¥,Total,2022,67.308338,65.170350,65.829296,66.874067,67.878400,68.424932,67.308338,66.844329
3,0,xtandi,DE,¥,Total,2022,518.145979,502.403626,506.973546,514.934678,522.738919,526.640954,518.132556,514.742387
4,0,xtandi,FR,¥,Total,2022,260.011082,251.834907,253.915031,257.698050,262.366899,264.591589,260.011082,258.021722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0,zolbe,FR,¥,Total,2025,2.234815,1.707674,1.782085,1.926181,2.070534,2.157552,1.933115,1.929744
176,0,zolbe,IT,¥,Total,2025,3.637843,2.798595,2.948740,3.198717,3.421893,3.556723,3.172199,3.184083
177,0,zolbe,ES,¥,Total,2025,1.740246,1.330294,1.394359,1.518672,1.637571,1.700793,1.515754,1.517027
178,0,zolbe,GB,¥,Total,2025,2.423964,1.866552,1.957543,2.117544,2.266085,2.365822,2.116121,2.113775


In [14]:
print(sum(output_df2[(output_df2['product'] =='zolbe')]['sales']))
print(sum(output_df2[(output_df2['product'] =='zolbe')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='zolbe')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='zolbe')]['sales_RA_75']))  

475.20551843110303
384.07843087736103
414.8990455927596
444.72223016865286


In [15]:
#############
# FEZO
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'fezo'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]
output_unc_df=output_unc_df[(output_unc_df['product']!=product)]

for year in years:
    print(year)
    
    # Get Fezo US values, to create proportion
    fezo_base=sales_ptrs[(sales_ptrs['product']==product) & (sales_ptrs['region']=='US') & (sales_ptrs['year']==year)& (sales_ptrs['indication']=='Total')]['sales_p1_RA']
    fezo_10 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc19'])
    fezo_25 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc20'])
    fezo_50 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc21'])
    fezo_75 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc22'])
    fezo_90 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc23'])
    
    for region in regions:
        # Get Product Worldwide sales by year (series) 
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year)]
        prod_ww_sales['sales_RA_10'] = float(fezo_10/fezo_base)*float(prod_ww_sales['sales_p1_RA'])
        prod_ww_sales['sales_RA_25'] = float(fezo_25/fezo_base)*float(prod_ww_sales['sales_p1_RA'])
        prod_ww_sales['sales_RA_50'] = float(fezo_50/fezo_base)*float(prod_ww_sales['sales_p1_RA'])
        prod_ww_sales['sales_RA_75'] = float(fezo_75/fezo_base)*float(prod_ww_sales['sales_p1_RA'])
        prod_ww_sales['sales_RA_90'] = float(fezo_90/fezo_base)*float(prod_ww_sales['sales_p1_RA'])
    
        prod_ww_sales['sales_p2_RA'] = prod_ww_sales['sales_RA_50']

        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)

output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,2375.790259,3334.078478,3405.793849,3469.879992,3505.241242,3408.553603,3222.432940
1,0,xtandi,JP,¥,Total,2022,438.632820,425.456861,429.309008,435.738449,441.815860,445.635243,438.632820,435.586629
2,0,xtandi,CN,¥,Total,2022,67.308338,65.170350,65.829296,66.874067,67.878400,68.424932,67.308338,66.844329
3,0,xtandi,DE,¥,Total,2022,518.145979,502.403626,506.973546,514.934678,522.738919,526.640954,518.132556,514.742387
4,0,xtandi,FR,¥,Total,2022,260.011082,251.834907,253.915031,257.698050,262.366899,264.591589,260.011082,258.021722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,0,fezo,FR,¥,Total,2025,14.440805,7.686611,9.624027,12.375386,16.152774,19.792593,14.195311,12.375386
212,0,fezo,IT,¥,Total,2025,12.874491,6.845915,8.571433,11.021873,14.386122,17.627849,12.642750,11.021873
213,0,fezo,ES,¥,Total,2025,5.675226,3.002394,3.759150,4.833832,6.309282,7.730997,5.544696,4.833832
214,0,fezo,GB,¥,Total,2025,17.030000,9.120128,11.418862,14.683338,19.165190,23.483819,16.842670,14.683338


In [16]:
print(sum(output_df2[(output_df2['product'] =='fezo')]['sales']))
print(sum(output_df2[(output_df2['product'] =='fezo')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='fezo')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='fezo')]['sales_RA_75']))   

5336.288938830225
3531.399456558151
4540.813215243884
5927.084555987782


In [17]:
#############
# MIRABEGRON
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'mirabegron'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for region in regions:
    n=sims
    sales_ra22 = []
    sales_ra23 = []
    sales_ra24 = []
    sales_ra25 = []
    
    for i in range(n):
        # Get Base Uncertainty
        unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
        unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

        # Get High,Base,Low Uncertainty
        unc_hbl_prob=np.random.randint(1,4)
        # unc_hbl_prob=1

        # Generate RA sales - High
        if (unc_hbl_prob==1):
            sales22=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2022) & (unc_pr['region'] ==region)]['unc24'])*float(1+unc_base_prob)
            sales23=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2023) & (unc_pr['region'] ==region)]['unc24'])*float(1+unc_base_prob)
            sales24=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2024) & (unc_pr['region'] ==region)]['unc24'])*float(1+unc_base_prob)
            sales25=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2025) & (unc_pr['region'] ==region)]['unc24'])*float(1+unc_base_prob)

        # Generate RA sales - Med
        if (unc_hbl_prob==2):
            sales22=float(sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2022) ]['sales'])*float(1+unc_base_prob)
            sales23=float(sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2023) ]['sales'])*float(1+unc_base_prob)
            sales24=float(sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2024) ]['sales'])*float(1+unc_base_prob)
            sales25=float(sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2025) ]['sales'])*float(1+unc_base_prob)
        
        # Generate RA sales - Low
        if (unc_hbl_prob==3):
            sales22=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2022) & (unc_pr['region'] ==region)]['unc25'])*float(1+unc_base_prob)
            sales23=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2023) & (unc_pr['region'] ==region)]['unc25'])*float(1+unc_base_prob)
            sales24=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2024) & (unc_pr['region'] ==region)]['unc25'])*float(1+unc_base_prob)
            sales25=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2025) & (unc_pr['region'] ==region)]['unc25'])*float(1+unc_base_prob)
        
        sales_ra22.append(sales22)
        sales_ra23.append(sales23)
        sales_ra24.append(sales24)
        sales_ra25.append(sales25)
 
     # Get Product, Region sales by year (series)
    prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2022) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra22, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra22, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra22, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra22, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra22, 90)
    prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra22)

    # Append to sales df
    output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2023) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra23, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra23, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra23, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra23, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra23, 90)
    prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra23)

    # Append to sales df
    output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2024) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra24, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra24, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra24, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra24, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra24, 90)
    prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra24)

    # Append to sales df
    output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2025) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra25, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra25, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra25, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra25, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra25, 90)
    prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra25)

    # Append to sales df
    output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,2375.790259,3334.078478,3405.793849,3469.879992,3505.241242,3408.553603,3222.432940
1,0,xtandi,JP,¥,Total,2022,438.632820,425.456861,429.309008,435.738449,441.815860,445.635243,438.632820,435.586629
2,0,xtandi,CN,¥,Total,2022,67.308338,65.170350,65.829296,66.874067,67.878400,68.424932,67.308338,66.844329
3,0,xtandi,DE,¥,Total,2022,518.145979,502.403626,506.973546,514.934678,522.738919,526.640954,518.132556,514.742387
4,0,xtandi,FR,¥,Total,2022,260.011082,251.834907,253.915031,257.698050,262.366899,264.591589,260.011082,258.021722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,mirabegron,GB,¥,Total,2025,69.956730,67.166213,68.020496,69.737398,71.423082,72.606899,69.956730,69.790003
248,0,mirabegron,WWex8,¥,Total,2022,306.367270,294.266784,299.022081,306.566271,313.584354,318.225859,306.367270,306.306906
249,0,mirabegron,WWex8,¥,Total,2023,303.232806,287.577601,292.661726,299.751522,306.930149,311.817261,303.232806,299.671757
250,0,mirabegron,WWex8,¥,Total,2024,290.662682,275.445929,280.377621,287.123480,294.025168,298.668022,290.662682,287.104572


In [18]:
print(sum(output_df2[(output_df2['product'] =='mirabegron')]['sales']))
print(sum(output_df2[(output_df2['product'] =='mirabegron')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='mirabegron')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='mirabegron')]['sales_RA_75']))  

6512.1
5130.585951411328
6467.927037490609
7117.462885801218


In [19]:
#############
# TACROLIMUS
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'tacrolimus'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []

        for i in range(n):
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)
            
            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob)
            sales_ra.append(unc_all)
            
        # Get Product Worldwide sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
    
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,2375.790259,3334.078478,3405.793849,3469.879992,3505.241242,3408.553603,3222.432940
1,0,xtandi,JP,¥,Total,2022,438.632820,425.456861,429.309008,435.738449,441.815860,445.635243,438.632820,435.586629
2,0,xtandi,CN,¥,Total,2022,67.308338,65.170350,65.829296,66.874067,67.878400,68.424932,67.308338,66.844329
3,0,xtandi,DE,¥,Total,2022,518.145979,502.403626,506.973546,514.934678,522.738919,526.640954,518.132556,514.742387
4,0,xtandi,FR,¥,Total,2022,260.011082,251.834907,253.915031,257.698050,262.366899,264.591589,260.011082,258.021722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,0,tacrolimus,FR,¥,Total,2025,123.608011,118.675964,120.384411,123.446558,126.690846,128.658630,123.608011,123.564282
284,0,tacrolimus,IT,¥,Total,2025,36.814557,35.392327,35.898630,36.811147,37.674184,38.269359,36.814557,36.801143
285,0,tacrolimus,ES,¥,Total,2025,59.892065,57.480736,58.512370,59.877884,61.446213,62.247129,59.892065,59.928295
286,0,tacrolimus,GB,¥,Total,2025,54.303272,52.139250,52.885747,54.347172,55.704055,56.458458,54.303272,54.322225


In [20]:
print(sum(output_df2[(output_df2['product'] =='tacrolimus')]['sales']))
print(sum(output_df2[(output_df2['product'] =='tacrolimus')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='tacrolimus')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='tacrolimus')]['sales_RA_75']))  

6736.2300000000005
6571.753756856795
6739.789595073862
6906.104466381308


In [21]:
#############
# LEXISCAN
#############
years = [2022, 2023, 2024, 2025]
product = 'lexiscan'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for year in years:
    print(year)
    n=sims
    sales_ra = []
    
    for i in range(n):
        # Get Base Uncertainty
        unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year)& (unc_pr['region'] =='US')]['unc_base']
        unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

        # Generate RA sales
        sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['year']==year) ]
        unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob)
        sales_ra.append(unc_all)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['year']==year) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
    prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)

    
    # Append to sales df
    output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,2375.790259,3334.078478,3405.793849,3469.879992,3505.241242,3408.553603,3222.432940
1,0,xtandi,JP,¥,Total,2022,438.632820,425.456861,429.309008,435.738449,441.815860,445.635243,438.632820,435.586629
2,0,xtandi,CN,¥,Total,2022,67.308338,65.170350,65.829296,66.874067,67.878400,68.424932,67.308338,66.844329
3,0,xtandi,DE,¥,Total,2022,518.145979,502.403626,506.973546,514.934678,522.738919,526.640954,518.132556,514.742387
4,0,xtandi,FR,¥,Total,2022,260.011082,251.834907,253.915031,257.698050,262.366899,264.591589,260.011082,258.021722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,0,tacrolimus,WWex8,¥,Total,2025,706.677430,679.309600,690.067377,707.362458,724.155432,735.774430,706.677430,707.278934
288,0,lexiscan,WW,¥,Total,2022,759.733936,729.427548,739.804541,757.398189,777.162886,789.370363,759.733936,758.341672
289,0,lexiscan,WW,¥,Total,2023,379.866968,364.473967,369.726859,378.987406,389.533738,394.770351,379.866968,379.381782
290,0,lexiscan,WW,¥,Total,2024,18.993348,18.216277,18.514985,18.986205,19.443227,19.742639,18.993348,18.990755


In [22]:
print(sum(output_df2[(output_df2['product'] =='lexiscan')]['sales']))
print(sum(output_df2[(output_df2['product'] =='lexiscan')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='lexiscan')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='lexiscan')]['sales_RA_75']))

1174.7385985710891
1143.8031500857076
1171.5397126269145
1202.702736669067


In [23]:
#############
# OTHERS
#############
years = [2022, 2023, 2024, 2025]
region = 'WW'
other_products = ['evenity','cresemba','tamsulosin','suglat','ambisome','vesicare','mycamine','sujanu','symraf','cimzia','blincyto','repatha','linzess',
'vesomni','myslee/stilnox','irribow','gonax','feburic','asamax','josamycin','non prod','istodax','other merchandise products','allelock','tr','acr','blz','bonoteo','seroquel','geninax','col','inf-v','kiklin',
'ofa','regnite','srs','st','dificlir','p_aco - acofide','other astellas products']

for product in other_products:
    # Clear output for rerun
    output_df2=output_df2[(output_df2['product']!=product)]
    print(product)
    for year in years:
        n=sims
        sales_ra = []
    
        for i in range(n):
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob)
            sales_ra.append(unc_all)

        # Get Product Worldwide sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
    

        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

evenity
cresemba
tamsulosin
suglat
ambisome
vesicare
mycamine
sujanu
symraf
cimzia
blincyto
repatha
linzess
vesomni
myslee/stilnox
irribow
gonax
feburic
asamax
josamycin
non prod
istodax
other merchandise products
allelock
tr
acr
blz
bonoteo
seroquel
geninax
col
inf-v
kiklin
ofa
regnite
srs
st
dificlir
p_aco - acofide
other astellas products


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,2375.790259,3334.078478,3405.793849,3469.879992,3505.241242,3408.553603,3222.432940
1,0,xtandi,JP,¥,Total,2022,438.632820,425.456861,429.309008,435.738449,441.815860,445.635243,438.632820,435.586629
2,0,xtandi,CN,¥,Total,2022,67.308338,65.170350,65.829296,66.874067,67.878400,68.424932,67.308338,66.844329
3,0,xtandi,DE,¥,Total,2022,518.145979,502.403626,506.973546,514.934678,522.738919,526.640954,518.132556,514.742387
4,0,xtandi,FR,¥,Total,2022,260.011082,251.834907,253.915031,257.698050,262.366899,264.591589,260.011082,258.021722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,0,p_aco - acofide,WW,¥,Total,2025,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
448,0,other astellas products,WW,¥,Total,2022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
449,0,other astellas products,WW,¥,Total,2023,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
450,0,other astellas products,WW,¥,Total,2024,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
print(sum(output_df2[(output_df2['product'].isin(other_products))]['sales']))
print(sum(output_df2[(output_df2['product'].isin(other_products))]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'].isin(other_products))]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'].isin(other_products))]['sales_RA_75']))

9797.389256678245
9309.67693222813
9797.216345669876
10282.480528454455


In [25]:
# Save output of Phase 2
output_df2.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\output_phase2.csv")

# Add WW sales to output
output_df3=output_df2
output_df3=output_df3.groupby(['product','units','indication','year']).sum().reset_index()
output_df3['region']='ALL'
output_df3 = output_df2.append(output_df3, ignore_index=True)

output_df3.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\output_phase2_agg.csv")

In [26]:
#############
# PHASE 3: ABOVE BRAND
#############
# Create output df
output_df4 = pd.DataFrame(columns=['tag','product', 'region', 'units', 'indication', 'year', 'sales', 'sales_RA_10', 'sales_RA_25','sales_RA_50','sales_RA_75', 'sales_RA_90', 'sales_p1_RA', 'sales_p2_RA', 'sales_p3_RA'])

# Inputs
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB']
products = ['evrenzo', 'fezo', 'mirabegron', 'padcev', 'tacrolimus', 'xospata', 'xtandi', 'zolbe']
sales_p3a=output_df2[(output_df2['region'].isin(regions))] # These go thru phase3
sales_p3b=output_df2[~(output_df2['region'].isin(regions))] # These do not go thru phase3

for product in products:
    print(product)

    for year in years:
    
        for region in regions:
            n=sims
            sales_ra = []
            unc_ab1_ra = []
            unc_ab2_ra = []
            unc_ab3_ra = []
            unc_ab4_ra = []

            for i in range(n):
                # Get Above Brand Uncertainty Probability (value)
                unc_ab1_prob=np.random.binomial(size=1, n=1, p=unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc_ab1')]['probability'])
                unc_ab2_prob=np.random.binomial(size=1, n=1, p=unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc_ab2')]['probability'])
                unc_ab3_prob=np.random.binomial(size=1, n=1, p=unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc_ab3')]['probability'])
                unc_ab4_prob=np.random.binomial(size=1, n=1, p=unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc_ab4')]['probability'])
                
                # Get sales from phase 2
                sales=sales_p3a[(sales_p3a['product'] ==product) & (sales_p3a['region']==region) & (sales_p3a['year']==year) ]
                
                # Get Uncertainty Quant by year (series)
                unc_ab1=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_ab1']*float(sales['sales_p2_RA'])*unc_ab1_prob
                unc_ab2=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_ab2']*float(sales['sales_p2_RA'])*unc_ab2_prob
                unc_ab3=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_ab3']*float(sales['sales_p2_RA'])*unc_ab3_prob
                unc_ab4=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_ab4']*float(sales['sales_p2_RA'])*unc_ab4_prob
                
                # Generate RA sales
                unc_all=float(sales['sales_p2_RA']) + unc_ab1 + unc_ab2 + unc_ab3 + unc_ab4
                sales_ra.append(unc_all)
                
                # Generate uncertainty arrays
                unc_ab1_ra.append(float(unc_ab1))
                unc_ab2_ra.append(float(unc_ab2))
                unc_ab3_ra.append(float(unc_ab3))
                unc_ab4_ra.append(float(unc_ab4))
            
            # Get Product Worldwide sales by year (series)
            prod_ww_sales=sales_p3a[(sales_p3a['product'] ==product) & (sales_p3a['region']==region) & (sales_p3a['year']==year) ]
            prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
            prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
            prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
            prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
            prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
            prod_ww_sales['sales_p3_RA'] = np.mean(sales_ra)
            
            # Add uncertainty arrays to output_unc
            output_unc=sales_p3a[(sales_p3a['product'] ==product) & (sales_p3a['region']==region) & (sales_p3a['year']==year) ][['tag', 'product', 'region', 'units', 'indication', 'year', 'sales','sales_p1_RA', 'sales_p2_RA']]
            output_unc['unc_ab1_ra'] = np.mean(unc_ab1_ra)
            output_unc['unc_ab2_ra'] = np.mean(unc_ab2_ra)
            output_unc['unc_ab3_ra'] = np.mean(unc_ab3_ra)
            output_unc['unc_ab4_ra'] = np.mean(unc_ab4_ra)

            # Append to sales df
            output_df4 = output_df4.append(prod_ww_sales, ignore_index=True)
            output_unc_df = output_unc_df.append(output_unc, ignore_index=True)
            
output_df4=output_df4.fillna(0)
output_df4

evrenzo
fezo
mirabegron
padcev
tacrolimus
xospata
xtandi
zolbe


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA,sales_p3_RA
0,0,evrenzo,US,¥,Total,2022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,evrenzo,JP,¥,Total,2022,55.767542,50.080645,50.080645,55.645161,55.645161,55.645161,55.767542,55.645161,52.896290
2,0,evrenzo,CN,¥,Total,2022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0,evrenzo,DE,¥,Total,2022,16.820283,14.234498,14.234498,14.234498,16.746468,16.746468,16.820283,16.746468,15.113688
4,0,evrenzo,FR,¥,Total,2022,0.696334,0.575768,0.575768,0.575768,0.677375,0.677375,0.696334,0.677375,0.612042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0,zolbe,DE,¥,Total,2025,24.608011,18.418513,18.418513,18.418513,21.668839,21.668839,21.753482,21.668839,19.556127
252,0,zolbe,FR,¥,Total,2025,2.234815,1.640282,1.640282,1.640282,1.929744,1.929744,1.933115,1.929744,1.743330
253,0,zolbe,IT,¥,Total,2025,3.637843,2.706471,2.706471,2.706471,3.184083,3.184083,3.172199,3.184083,2.872202
254,0,zolbe,ES,¥,Total,2025,1.740246,1.289473,1.289473,1.289473,1.517027,1.517027,1.515754,1.517027,1.371164


In [27]:
# Update output uncertainty df for PTRS
output_unc_df['unc_ptrs'] = output_unc_df['sales_p1_RA'] - output_unc_df['sales']

# Update output uncertainty df for PTRS
output_unc_df=output_unc_df.fillna(0)

# Select columns needed
output_unc_df2=output_unc_df[['tag', 'product', 'region', 'units', 'indication', 'year', 'sales',
                             'unc_ptrs','unc1_ra', 'unc2_ra', 'unc3_ra', 'unc4_ra',
                             'unc5_ra', 'unc6_ra', 'unc7_ra', 'unc8_ra', 'unc9_ra', 'unc10_ra',
                             'unc11_ra', 'unc12_ra', 'unc13_ra', 'unc14_ra', 'unc15_ra', 'unc16_ra',
                             'unc17_ra', 'unc18_ra', 'unc26_ra', 
                             'unc_ab1_ra', 'unc_ab2_ra','unc_ab3_ra', 'unc_ab4_ra']]

# Pivot table
id_vars=['tag', 'product', 'region', 'units', 'indication', 'year', 'sales']
value_vars=['unc_ptrs','unc1_ra', 'unc2_ra', 'unc3_ra', 'unc4_ra',
            'unc5_ra', 'unc6_ra', 'unc7_ra', 'unc8_ra', 'unc9_ra', 'unc10_ra',
            'unc11_ra', 'unc12_ra', 'unc13_ra', 'unc14_ra', 'unc15_ra', 'unc16_ra',
            'unc17_ra', 'unc18_ra', 'unc26_ra', 
            'unc_ab1_ra', 'unc_ab2_ra','unc_ab3_ra', 'unc_ab4_ra']

output_unc_df2=pd.melt(output_unc_df2, id_vars=id_vars, value_vars=value_vars)

# # Merge unmcertainty name to output_unc_df2
# output_unc_df2=output_unc_df2.merge(unc_pr_desc, how='left', left_on='lkey', right_on='rkey')

output_unc_df2.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\output_unc_df.csv")

In [28]:
# Add back tags that did not go through phase 3
sales_p3b['sales_p3_RA']=sales_p3b['sales_p2_RA']
output_phase3=pd.concat([output_df4, sales_p3b], ignore_index=True, axis=0)

# Save output of Phase 3
output_phase3.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\output_phase3.csv")

# Add WW sales to output
output_phase3_agg=output_phase3
output_phase3_agg=output_phase3_agg.groupby(['product','units','indication','year']).sum().reset_index()
output_phase3_agg['region']='ALL'
output_phase3_agg = output_phase3.append(output_phase3_agg, ignore_index=True)

output_phase3_agg.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\output_phase3_agg.csv")

In [29]:
######################
# METADATA
######################
sim_end = datetime.now()
td =  sim_end - sim_start
td_mins = int(round(td.total_seconds() / 60))
metatdata = {'sim_start': sim_start, 'sim_end': sim_end, 'sims_run': sims, 'sim_time': td_mins}
metatdata_df=pd.DataFrame.from_dict(metatdata, orient='index')
metatdata_df.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\metatdata.csv")